In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import pandas as pd
from glob import glob
from tqdm import tqdm
from unidecode import unidecode
import random
import json

In [3]:
base_directory = '/home/husein/speech-bahasa'

In [4]:
with open(f'{base_directory}/CHANNEL0/SCRIPT/FEMALE_01.txt') as fopen:
    texts = fopen.read().split('\n')

singlishs = []
for t in texts:
    splitted = t.split('\t')
    if len(splitted) == 2:
        singlishs.append((f'{base_directory}/CHANNEL0/WAVE/FEMALE_01/{splitted[0]}.wav', splitted[1]))
        
len(singlishs)

6033

In [5]:
wave_texts = glob('WAVE-text/*.TXT')

for f in tqdm(wave_texts):
    speaker = f.split('/')[1].replace('.TXT', '')
    channel = speaker[-1]
    speaker = speaker[1:-1]
    
    with open(f) as fopen:
        texts = list(filter(None, fopen.read().split('\n')))[::2]
    
    for text in texts:
        splitted = text.split('\t')
        wav = unidecode(splitted[0])
        path = f'{base_directory}/WAVE/SPEAKER{speaker}/SESSION{channel}/{wav}.WAV'
        
        if os.path.exists(path) and len(splitted[1]):
            singlishs.append((path, splitted[1]))
        else:
            print(splitted, path)
            pass

100%|██████████| 2034/2034 [00:07<00:00, 271.89it/s]


In [6]:
singlish = glob(f'{base_directory}/part3-splitted/wav/*.wav')
len(singlish)

1261328

In [7]:
def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [w for w in string if w[0] != '<' and w[-1] != '>']
    return ' '.join(string)

for i in tqdm(singlish):
    try:
        p = i.replace('/wav','/text')
        with open(f'{p}.txt') as fopen:
            text = fopen.read()
        if len(text) < 2:
            continue
        if text[0] == '<' and text[-1] == '>':
            continue
        text = replace_paralinguistic(text)
        singlishs.append((i, text))
    except Exception as e:
        print(e)

100%|██████████| 1261328/1261328 [04:14<00:00, 4948.40it/s]


In [8]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
        
    string = unicodedata.normalize('NFC', string.lower())
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [9]:
audios = singlishs
audios, texts = zip(*audios)

In [10]:
len(texts)

1466595

In [11]:
processed_text = [preprocessing_text(t) for t in tqdm(texts)]

100%|██████████| 1466595/1466595 [00:50<00:00, 29121.91it/s]


In [12]:
import malaya_speech

In [13]:
tokenizer = malaya_speech.subword.generate_tokenizer(processed_text, max_subword_length = 3)

In [15]:
malaya_speech.subword.save(tokenizer, 'transducer-singlish.subword')